Needed Imports

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import imgaug as ia
# import imgaug.augmenters as iaa
import warnings
warnings.filterwarnings("ignore")
ia.seed(1)
from imgaug import augmenters as iaa 
import re
import imageio

In [ ]:
import glob 
import cv2 as cv
import os
import scipy.io
import datetime

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline 


In [ ]:
#move wiki.mat file to current directory
mv wiki/wiki.mat .

In [ ]:
## Convert Mat File TO csv

In [ ]:
def convert_mat_tocsv():
    mat = scipy.io.loadmat('wiki.mat')
    instances = mat['wiki'][0][0][0].shape[1]
    columns = ["dob", "photo_taken", "full_path", "gender",\
                "name", "face_location", "face_score", "second_face_score"]
    df = pd.DataFrame(index = range(0,instances), columns = columns)
    for i in mat:
        if i == "wiki":
            current_array = mat[i][0][0]
            for j in range(len(current_array)):
                df[columns[j]] = pd.DataFrame(current_array[j][0])
    return df

df=convert_mat_tocsv()


In [ ]:
#
aug = iaa.SomeOf(2, [
    iaa.Affine(rotate=45),
    iaa.AdditiveGaussianNoise(scale=0.2*255),
    iaa.Add(50, per_channel=True),
    iaa.Sharpen(alpha=0.5),
    iaa.AddToHueAndSaturation((-60, 60)),
])    

In [ ]:
#Path For Image Augmentation
path_for_aug='wiki'

In [ ]:
#We are Augmenting Images to Avoid Overfitting (Data Has Nearly 49000 MAle Pictures and 12000 Female Pictures)
#This Section Is Really Time Consuming!!
def AugmentImages(aug_Path,augmentator):
    #Creating New Dataframe With Same Columns of Original.
    dff=pd.DataFrame(columns=["dob", "photo_taken", "full_path", "gender",\
                  "name", "face_location", "face_score", "second_face_score"])
    #Since The Dataframe shows that Dataset is unbalanced we need to augment only The Images that have 0.0 gender(Female Images)
    for index,image in enumerate(df['full_path']):
        #getting The Gender of Image
        gend=df.loc[df['full_path'] == image[0], 'gender'].iloc[0]
        #if Gender is Female
        if (gend==0.0).any():
            #Augment The image 3 times
            for i in range(3):
                try:
                    #get The Path of The image
                    img_path=os.path.join(aug_Path,image[0])
                    #Read Image
                    img=imageio.imread(img_path)
                    #augment it With Allready Chosen augmentation Types
                    aug=augmentator(image=img)
                    #get The Folder OF the image
                    folder=(img_path).split('/',4)[1]
                    #get The name of the image
                    imagename = os.path.basename(img_path)
                    #add 'aug' in the middle of the name(its Arbitrary, i Just added it in the middle)
                    aug_name=f"{imagename[:4]}aug{imagename[:-4]}"
                    #write augmented Images in The same folder 
                    cv.imwrite(f'wiki/{folder}/{aug_name}{index}{i}.jpg',aug)
                    #Get All column Values of the image
                    gender=df.loc[df['full_path'] == image[0], 'gender'].iloc[0]
                    face_score=df.loc[df['full_path'] == image[0], 'face_score'].iloc[0]
                    second_face_score=df.loc[df['full_path'] == image[0], 'second_face_score'].iloc[0]
                    name=df.loc[df['full_path'] == image[0], 'name'].iloc[0]
                    photo_taken=df.loc[df['full_path'] == image[0], 'photo_taken'].iloc[0]
                    dob=df.loc[df['full_path'] == image[0], 'dob'].iloc[0]
                    face_location=df.loc[df['full_path'] == image[0], 'face_location'].iloc[0]
                    #add The vales to new Dataframe
                    dff.loc[int(f'{index}{i}')]=[dob,photo_taken,[f'{folder}/{aug_name}{index}{i}.jpg'],gender,name,face_location,face_score,second_face_score]
                except:
                    continue  

    return dff

img=augment(path_for_aug,aug)

In [ ]:
#Concatenate Old And New Dataframe with new images, Set ignore_index To True to reset all the indices from zero to length of the images
all_df=pd.concat([df,img],ignore_index=True)

In [ ]:
#Create Face Directory To Save Cropped Faces
dirName = 'Face'
try:
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

In [ ]:
class Images(Dataset):
    def __init__(self,path,df,path1,transform=None):
        self.path=path
        self.df=df
        self.path1=path1
        self.transform=transform
        #run in Once
        self.export()

    #Crop Faces
    def cropFace(self,image,imgname):
        try:
            #Crop Faces with Haaras cascade
            facedata = "haarcascade_frontalface_default.xml"
            cascade = cv.CascadeClassifier(facedata)
            img = cv.imread(image)
            minisize = (img.shape[1],img.shape[0])
            miniframe = cv.resize(img, minisize)
            faces = cascade.detectMultiScale(miniframe)
            for f in faces:
                x, y, w, h = [ v for v in f ]
                cv.rectangle(img, (x,y), (x+w,y+h), (255,0,0))

                sub_face = img[y:y+h, x:x+w]
                face_file_name = f"Face/{imgname}.jpg"
                cv.imwrite(face_file_name, sub_face)
        except:
            del image

    #Export Cropped Faces To Face Directory 
    def export(self):
        images = glob.glob(self.path)
        for imgpath in images[0:]:
            imagename = os.path.basename(imgpath)
            imgname = os.path.splitext(imagename)[0]
            self.cropFace(imgpath, imgname)
    
        

    def __getitem__(self,index):
        #get the image with y_label Wich is Gender in this Case
        try:
            img_path=os.path.join(self.path1,self.df.iloc[index,2][0].split('/',1)[1])
            global img
            img=cv.imread(img_path,cv.IMREAD_GRAYSCALE)
            global y_label
            y_label=torch.tensor(int(self.df.iloc[index,3]))
            #Resize Images SInce they have to be Same Sizes
            img=cv.resize(img,(132,132))
            img=torch.tensor(img)
        except:
            img=torch.tensor(np.ones((132,132)))      
    
        return (img.float(),y_label)


    #returns The Quantity Of Images
    def __len__(self):
        return len(self.df)



dataset=Images(path,all_df,path1)

In [ ]:
#Split Dataset Into Train,validation And Test Sets
train_set,val_set,test_set=torch.utils.data.random_split(dataset,[int(len(dataset)*0.7),int(len(dataset)*0.15),int(len(dataset)*0.15+1)])
train_loader=DataLoader(train_set,batch_size=64,shuffle=True)
test_loader=DataLoader(test_set,batch_size=64,shuffle=False)
val_loader=DataLoader(val_set,batch_size=64,shuffle=False)


In [ ]:
class CnnImages(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,5,1)
        self.conv2=nn.Conv2d(6,16,5,1)
        self.conv3=nn.Conv2d(16,10,5,1)
        self.conv4=nn.Conv2d(10,6,5,1)
        self.fc1=nn.Linear(6*4*4,84)
        self.fc2=nn.Linear(84,10)
        self.fc3=nn.Linear(10,2)

  

    def forward(self,X):
        X=X.unsqueeze(1)
        X=F.relu(self.conv1(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv2(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv3(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv4(X))
        X=F.max_pool2d(X,2,2)
        X=X.view(-1,6*4*4)
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=self.fc3(X)
        return F.log_softmax(X,dim=1) 



In [ ]:
torch.manual_seed(42)
model=CnnImages()
model

In [ ]:
#I Chose CrossEntropyLoss Because i think its Optimal for binary classification, but you can try Other Loss Functions too
criterion=nn.CrossEntropyLoss()
#If you Lower The Learning Rate i would recommend To increase Epochs, in opposite, if you raise learning rate i would recommend to lower the epochs
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
import time 
start_time=time.time()

epochs=20
test_losses=[]
train_losses=[]
train_correct=[]
test_correct=[]

for i in range(epochs):
    
    trn_corr=0
    tst_corr=0

    for b,(X_train,y_train) in enumerate(train_loader):
        b+=1

        y_pred=model(X_train)
        loss=criterion(y_pred,y_train)
        #get the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr

        #update Parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if b%1000 == 0:
            print(f'epoch: {i:2} loss: {loss.item():10.8f}  \
    accuracy: {trn_corr.item()*100/(64*b):7.3f}%')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    with torch.no_grad():
        for b,(X_test,y_test) in enumerate(val_loader):
            y_val=model(X_test)
            predicted=torch.max(y_val.data,1)[1]
            tst_corr+=(predicted==y_test).sum()
        loss=criterion(y_val,y_test)
        test_losses.append(loss)
        test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds')            



In [ ]:
plt.plot([t/64 for t in train_correct], label='training accuracy')
plt.plot([t/64 for t in test_correct],label='test Accuracy')
plt.title('Accuracy at the end of each epoch')   

In [ ]:
#Change Image index To test Different Images
class_names=['female','male']
image_index=0
model.eval()

with torch.no_grad():
    new_pred = model(test_data[image_index][0].view(1,1,132,132)).argmax()
    
class_names[new_pred.item()]